# tensorflow v2.xx에서 v1 사용하기

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() #tensorflow v2 비활성화하고 v1 활성화
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


## tensorflow
 - 데이터 흐름 그래프 (tensor 객체의 흐름)을 사용하는 수치 계산 라이브러리
 - 그래프는 node(연산)과 edge로 구성
 - sess = tf.Sessio()을 이용해서 실행환경
 - sess.run()을 통해서 실행 결과를 확인

In [2]:
#1단계 : tensor 정의
node1 = tf.constant ('Hello, Tensorflow')

#2단계 : 세션(연산을 실행하는 환경) 생성
sess = tf.Session()
print(node1)

#3단계 : 실제 실행 및 출력
print(sess.run(node1))
print(sess.run(node1).decode())

Tensor("Const:0", shape=(), dtype=string)
b'Hello, Tensorflow'
Hello, Tensorflow


In [3]:
#간단한 연산  tensor graph
#1. 그래프 정의
node1 = tf.constant(10,dtype=tf.float16)
node2 = tf.constant(20,dtype=tf.float16)
node3 = tf.add(node1, node2)

#2. 세션 생성
sess = tf.Session()

#3. 세션 실행 & 결과
print(sess.run([node1, node2, node3]))

[10.0, 20.0, 30.0]


In [4]:
# 타입 변경
import numpy as np
node1 = tf.constant(np.array([1,2,3]), dtype=tf.int16)
node2 = tf.cast(node1, dtype=tf.float32)
sess = tf.Session()
print(sess.run([node1, node2]))

[array([1, 2, 3], dtype=int16), array([1., 2., 3.], dtype=float32)]


In [5]:
#평균값 계산 : tf.reduce_mean()
data = np.array([1.,2.,3.,4.])
m = tf.reduce_mean(data)
sess = tf.Session()
print(sess.run(m))

2.5


In [6]:
# tf.random_normal([size]) : 평균이 0이고 표준편차가 1인 난수 size개 발생
w = tf.random_normal([3]) 
sess = tf.Session()
sess.run(w)

array([0.8348411, 0.3869306, 1.6751621], dtype=float32)

In [7]:
# 변수노드
w = tf.Variable(tf.random_normal([1]))
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(w)

array([-1.1580836], dtype=float32)

# Tensorflow v1을 이용한 회귀분석 구현

## 독립변수 x가 1개, 타겟(종속)변수 y 가 1개

In [8]:
# define tensor graph

#dataset 확보
x = np.array([1,2,3])
y = np.array([2,3,4])

# weight와 bias
W = tf.Variable(tf.random.normal([1]), name = 'weight')
b = tf.Variable(tf.random.normal([1]), name = 'bias')

#Hat, Hypothesis : 결과는 numpy 배열
H = W*x+b

#cost function : 손실함수 :mse
cost = tf.reduce_mean(tf.square(H-y))

'''
학습목적 : cost가 최소가 되는 W,b를 찾는 것
cost 함수는 2차 함수이므로, 곡선그래프, 곡선위 미분값이 0이 되는 방향
(경사하강법 Gradient Descent)
'''

optimizer = tf.train.GradientDescentOptimizer(learning_rate =0.01)
train = optimizer.minimize(cost)

#세션 생성
sess = tf.Session()

#W,b 초기화
sess.run(tf.global_variables_initializer())

#6000번 학습 (v2에서의 fit함수)
for step in range(6001):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b])
    if step%300==0:
        print("{}번째 cost: {}, w:{}, b:{}".format(step,cost_val,W_val, b_val))

0번째 cost: 4.332727432250977, w:[0.4883755], b:[0.21861726]
300번째 cost: 0.007389886770397425, w:[1.0996023], b:[0.77358043]
600번째 cost: 0.0017437107162550092, w:[1.0483825], b:[0.8900153]
900번째 cost: 0.0004114441981073469, w:[1.0235021], b:[0.9465744]
1200번째 cost: 9.7080715931952e-05, w:[1.0114161], b:[0.97404855]
1500번째 cost: 2.290635893587023e-05, w:[1.0055454], b:[0.98739415]
1800번째 cost: 5.4051911320129875e-06, w:[1.0026938], b:[0.9938764]
2100번째 cost: 1.2759128367179073e-06, w:[1.0013088], b:[0.99702495]
2400번째 cost: 3.015577760834276e-07, w:[1.0006365], b:[0.9985537]
2700번째 cost: 7.144279123849628e-08, w:[1.00031], b:[0.99929607]
3000번째 cost: 1.7025277188054133e-08, w:[1.0001516], b:[0.99965644]
3300번째 cost: 4.0660892430821605e-09, w:[1.0000738], b:[0.99983215]
3600번째 cost: 9.778062803889043e-10, w:[1.0000361], b:[0.9999175]
3900번째 cost: 2.280368532892041e-10, w:[1.0000176], b:[0.9999603]
4200번째 cost: 6.850934403113129e-11, w:[1.0000099], b:[0.9999783]
4500번째 cost: 5.3284782836060

In [9]:
# 최종적으로 나온 회귀식 H = W*x + b
W_,b_ = sess.run([W,b])
W_,b_

(array([1.0000087], dtype=float32), array([0.9999811], dtype=float32))

In [10]:
def predict(x):
    return W_[0] * x+b_[0]

In [11]:
predict(5)

6.000024616718292

## predict을 위한 placeholder 이용
 - placeholder : 외부에서 데이터를 입력받을 수 있는 노드

In [12]:
x = tf.placeholder(dtype=tf.float32)
H = 1*x +1

sess = tf.Session()
sess.run([x, H], feed_dict={x : np.array([40,50])})

[array([40., 50.], dtype=float32), array([41., 51.], dtype=float32)]

In [13]:
# tensor 그래프 정의
# 데이터 셋 확보
x_data = np.array([1,2,3])
y_data = np.array([2,3,4])
# placeholder 설정(x, y)
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight와 bias
W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')
# Hat, Hypothesis : 결과는 numpy 배열
H = W * x + b
# cost function (손실함수 : mse )
cost = tf.reduce_mean(tf.square(H-y))
'''
학습목적 : cost가 최소가 되는 W, b를 찾는 것
cost 함수는 2차함수이므로 곡선 그래프. 곡선위 미분값이 0이 되는 방향
(경사하강법GradientDescent)
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# 세션 생성
sess = tf.Session()
# W, b 초기화
sess.run(tf.global_variables_initializer())
# 6000번 학습 (v2 에서의 fit함수)
for step in range(6001):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b],
                                        feed_dict={x:x_data,
                                                   y:y_data})
    if step%300==0:
        print("{}번째 cost:{}, W:{}, b:{}".format(step, cost_val, W_val, b_val))

0번째 cost:13.233242988586426, W:[-0.03043158], b:[-0.06324004]
300번째 cost:0.008883467875421047, W:[1.1092049], b:[0.75175166]
600번째 cost:0.00209613423794508, W:[1.0530468], b:[0.8794121]
900번째 cost:0.0004946073167957366, W:[1.025768], b:[0.9414233]
1200번째 cost:0.00011670589447021484, W:[1.0125169], b:[0.9715463]
1500번째 cost:2.753807530098129e-05, W:[1.0060803], b:[0.9861783]
1800번째 cost:6.497823051176965e-06, W:[1.0029534], b:[0.993286]
2100번째 cost:1.5337749346144847e-06, W:[1.0014352], b:[0.9967379]
2400번째 cost:3.626595059813553e-07, W:[1.000698], b:[0.99841404]
2700번째 cost:8.590239986006054e-08, W:[1.0003399], b:[0.99922794]
3000번째 cost:2.042374980248951e-08, W:[1.0001657], b:[0.9996237]
3300번째 cost:4.883001114563967e-09, W:[1.0000808], b:[0.99981594]
3600번째 cost:1.2105753066649072e-09, W:[1.0000399], b:[0.9999085]
3900번째 cost:2.77585371355471e-10, W:[1.0000196], b:[0.999956]
4200번째 cost:8.621251629259419e-11, W:[1.0000108], b:[0.99997604]
4500번째 cost:5.195962063386794e-11, W:[1.00000

In [14]:
#예측하기
sess.run(H, feed_dict={x:5})

array([6.0000243], dtype=float32)

## scale이 다른 데이터들의 회귀분석 구현(scale 조정 x)

In [15]:
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])

#placeholder 설정
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)

#weight & bias
W = tf.Variable(tf.random.normal([1]))
b = tf.Variable(tf.random.normal([1]))

#Hypothesis
H = W * x + b

# cost (손실함수)
cost = tf.reduce_mean(tf.square(H-y))

# 경사하강법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cost)

#Session & 변수 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#학습
for step in range (1, 6001):
    _, cost_val = sess.run([train, cost],feed_dict={x:x_data, y:y_data})
    if step%300==0:
        print("{}/6000번째 cost:{}".format(step+1, cost_val))

301/6000번째 cost:115.57442474365234
601/6000번째 cost:79.87027740478516
901/6000번째 cost:79.5494613647461
1201/6000번째 cost:79.53256225585938
1501/6000번째 cost:79.51873779296875
1801/6000번째 cost:79.50544738769531
2101/6000번째 cost:79.4925765991211
2401/6000번째 cost:79.48020935058594
2701/6000번째 cost:79.46826171875
3001/6000번째 cost:79.45671081542969
3301/6000번째 cost:79.44558715820312
3601/6000번째 cost:79.43485260009766
3901/6000번째 cost:79.42446899414062
4201/6000번째 cost:79.41449737548828
4501/6000번째 cost:79.40483093261719
4801/6000번째 cost:79.39552307128906
5101/6000번째 cost:79.38652038574219
5401/6000번째 cost:79.37786865234375
5701/6000번째 cost:79.36952209472656
6001/6000번째 cost:79.36144256591797


## scale이 다른 데이터들의 회귀분석 구현 (sscale 조정 o)
### scale 조정방법:  모든 데이터를 일정 범위 내로 조정

 - normalization (정규화) : 모든 데이터를 0-1 사이로 조정
 
                        x - x_min
    normalization = ----------------
                      x_max - x_min
                      
    * 위의 식을 써도 되지만 라이브러리(MinMaxScaler)을 주로 씀
    *위의 식보다 라이브러리 추천(sklearn.preprocessing.MinMaxScaler)

 - standardization (표준화) : 데이터의 평균을 0, 표준편차를 1로 조정
 
                        x - x_mean
    standardization = ----------------
                        xstd(표준편차)
         *위의 식보다 라이브러리 추천(sklearn.preprocessing.StandardScaler)

In [20]:

# 라이브러리 쓰지 않고 정규화
x_data = np.array([1, 2, 5, 8, 10])
y_data = np.array([5, 15, 68, 80, 95])
# x_data.min(), x_data.max(), x_data.mean(), x_data.std()
norm_scaled_x_data = (x_data - x_data.min()) / (x_data.max() - x_data.min())
norm_scaled_y_data = (y_data - y_data.min()) / (y_data.max() - y_data.min())

In [21]:
# 라이브러리를 사용하여 정규화
x_data = np.array([1, 2, 5, 8, 10]).reshape(-1,1)
y_data = np.array([5, 15, 68, 80, 95]).reshape(-1,1)
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler() # 독립변수 x를 정규화시킬 객체
scaler_x.fit(x_data)
norm_scaled_x_data = scaler_x.transform(x_data)
scaler_y = MinMaxScaler() # 종속(타겟)변수 y를 정규화시킬 객체
norm_scaled_y_data = scaler_y.fit_transform(y_data)
norm_scaled_y_data

array([[0.        ],
       [0.11111111],
       [0.7       ],
       [0.83333333],
       [1.        ]])

In [24]:
#라이브러리 쓰지 않고 표준화
x_data = np.array([1, 2, 5, 8, 10])
y_data = np.array([5, 15, 68, 80, 95])

stan_scaled_x_data = (x_data - x_data.mean()) / x_data.std()
stan_scaled_y_data = (y_data - y_data.mean()) / y_data.std()

np.column_stack([x_data, norm_scaled_x_data, stan_scaled_x_data])

array([[ 1.        ,  0.        , -1.22474487],
       [ 2.        ,  0.11111111, -0.93313895],
       [ 5.        ,  0.44444444, -0.05832118],
       [ 8.        ,  0.77777778,  0.81649658],
       [10.        ,  1.        ,  1.39970842]])

In [25]:
np.column_stack([y_data, norm_scaled_y_data, stan_scaled_y_data])

array([[ 5.        ,  0.        , -1.32373476],
       [15.        ,  0.11111111, -1.04563922],
       [68.        ,  0.7       ,  0.42826713],
       [80.        ,  0.83333333,  0.76198177],
       [95.        ,  1.        ,  1.17912508]])

In [27]:
#라이브러리를 사용하여 표준화
x_data = np.array([1, 2, 5, 8, 10]).reshape(-1,1)
y_data = np.array([5, 15, 68, 80, 95]).reshape(-1,1)

from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
stan_scaled_x_data = scaler_x.fit_transform(x_data)
scaler_y = StandardScaler()
stan_scaled_y_data = scaler_y.fit_transform(y_data)

np.column_stack([y_data, norm_scaled_y_data, stan_scaled_y_data])

array([[ 5.        ,  0.        , -1.32373476],
       [15.        ,  0.11111111, -1.04563922],
       [68.        ,  0.7       ,  0.42826713],
       [80.        ,  0.83333333,  0.76198177],
       [95.        ,  1.        ,  1.17912508]])

In [28]:
# 스케일_조정된 데이터를 다시 복구 :inverse_transform() 이용
scaler_y.inverse_transform(stan_scaled_y_data)

array([[ 5.],
       [15.],
       [68.],
       [80.],
       [95.]])

In [31]:
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])

#placeholder 설정
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)

#weight & bias
W = tf.Variable(tf.random.normal([1]))
b = tf.Variable(tf.random.normal([1]))

#Hypothesis
H = W * x + b

# cost (손실함수)
cost = tf.reduce_mean(tf.square(H-y))

# 경사하강법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train = optimizer.minimize(cost)

#Session & 변수 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#학습
for step in range (1, 6001):
    _, cost_val = sess.run([train, cost],feed_dict={x:norm_scaled_x_data,
                                                    y:norm_scaled_y_data})
    if step%300==0:
        print("{}/6000번째 cost:{}".format(step, cost_val))

300/6000번째 cost:0.025013167411088943
600/6000번째 cost:0.015524720773100853
900/6000번째 cost:0.01309670228511095
1200/6000번째 cost:0.01228755246847868
1500/6000번째 cost:0.011875261552631855
1800/6000번째 cost:0.011581889353692532
2100/6000번째 cost:0.011341358534991741
2400/6000번째 cost:0.011135460808873177
2700/6000번째 cost:0.010957158170640469
3000/6000번째 cost:0.01080227829515934
3300/6000번째 cost:0.010667642578482628
3600/6000번째 cost:0.010550577193498611
3900/6000번째 cost:0.010448782704770565
4200/6000번째 cost:0.010360272601246834
4500/6000번째 cost:0.010283303447067738
4800/6000번째 cost:0.010216385126113892
5100/6000번째 cost:0.01015818864107132
5400/6000번째 cost:0.01010759174823761
5700/6000번째 cost:0.010063589550554752
6000/6000번째 cost:0.010025328025221825
